# 因子分析模型(241)

## 因子分析模型

$$X=\mu+\Lambda F+\epsilon$$
其中$E(F)=0,\quad E(\epsilon)=0,\quad Cov(F)=I_m,\quad D(\epsilon)=Cov(\epsilon)=diag(\sigma_1^2,\cdots,\sigma_m^2),\quad Cov(F, \epsilon)=0$

- 原始变量$X$的协方差矩阵分解: $$Cov(X)=\Lambda\Lambda^T+diag(\sigma_1^2,\cdots,\sigma_m^2)$$

- 载荷因子$\alpha_{ij}$反映第$i$个变量和第$j$个公共因子的相关系数. 绝对值越大相关的密切程度越高.

- 变量$X_i$的共同度记为$h_i^2=\sum\limits_{j=1}^m\alpha_{ij}^2$, 又有$1=h_i^2+\sigma_i^2$, 故$h_i^2$越接近1, 因子分析效果越好

- $\Lambda$中各列平方和$S_j=\sum\limits_{i=1}^p\alpha_{ij}^2$, 用于衡量$F_j$的相对重要性.

## 主成分分析法估计载荷因子

设相关系数矩阵$R$的特征值和对应特征向量分别为: $\lambda_1\ge\lambda_2\ge\cdots\ge\lambda_p$和$\eta_1,\eta_2,\cdots,\eta_p$  
设m<p, 则因子载荷矩阵$\Lambda=[\sqrt{\lambda_1}\eta_1,\sqrt{\lambda_2}\eta_2,\cdots,\sqrt{\lambda_m}\eta_m]$  
特殊因子的方差用$R-\Lambda\Lambda^T$的对角元来估计. 即$\sigma_i^2=1-\sum\limits_{j=1}^m\alpha_{ij}^2$  

## 因子载荷矩阵的估计方法
1. 主成分分析法(242页);
通过因子旋转来直观的判断因子的实际意义
因子得分: 反过来把公共因子表示为原变量的线性组合.
因子得分函数: $F_j=c_j+\beta_{j1}X_1+\cdots+\beta_{jp}X_p,\ j=1,2,\cdots,m$

    - 因子得分:
        - 巴特莱特因子得分估计: $\hat{F}=(\Lambda^TD^{-1}\Lambda)^{-1}\Lambda^TD^{-1}(X-\mu)$
        - 回归方法因子得分估计: $\hat{F}=(\hat{F}_{ij})_{n\times m}=X_0R^{-1}\Lambda$

In [109]:
import numpy as np
from sklearn.decomposition import PCA
import scipy
import sympy
import pandas as pd

R = np.array([[1.000, 0.577, 0.509, 0.387, 0.462],
              [0.577, 1.000, 0.599, 0.389, 0.322],
              [0.509, 0.599, 1.000, 0.436, 0.426],
              [0.387, 0.389, 0.436, 1.000, 0.523],
              [0.462, 0.322, 0.426, 0.523, 1.000]
             ])

# 列为单位特征向量. 由于R是对称阵, 因此都是正交的特征向量(下面一行可以验证这一点).
# print(np.array([[np.round(np.sum(eigvec[:,i]*eigvec[:,j])) for i in range(R.shape[0])] for j in range(R.shape[1])]))
eigval, eigvec = np.linalg.eig(R)
order = eigval.argsort()[::-1]
eigvec = np.array([eigvec[:, order[i]] for i in range(order.shape[0])]).T
eigval = np.sort(eigval)[::-1]
eigvec = eigvec*np.sign(np.sum(eigvec, axis=0))
# 因子载荷矩阵
Lambda = eigvec*np.sqrt(eigval)
print(eigval, Lambda, sep='\n')

# 信息贡献率
b = np.array([eigval[i]/eigval.sum() for i in range(eigval.shape[0])])
print(b)
# 累积贡献率
alpha = np.array([b[:i+1].sum() for i in range(b.shape[0])])
print(alpha)

m = 2
# 特殊因子方差
var_e = [1-Lambda[i, :m].sum() for i in range(Lambda.shape[0])]
print(var_e)


[2.85671099 0.80916372 0.53967524 0.4515001  0.34294995]
[[ 0.78357657 -0.21619342 -0.44937468  0.25979429 -0.26426783]
 [ 0.77259486 -0.45813757  0.13090262  0.13873788  0.39600941]
 [ 0.79468181 -0.23428242  0.24614116 -0.44512148 -0.23425194]
 [ 0.71234149  0.47285413  0.39725834  0.31715858 -0.10283392]
 [ 0.71194547  0.52350245 -0.31969123 -0.25697497  0.22547776]]
[0.5713422  0.16183274 0.10793505 0.09030002 0.06858999]
[0.5713422  0.73317494 0.84110999 0.93141001 1.        ]
[0.4326168480911273, 0.6855427113216124, 0.43960060701853165, -0.1851956198795388, -0.23544791397276743]


In [39]:
from sklearn.datasets import load_digits
from sklearn.decomposition import FactorAnalysis
import numpy as np

X, _ = load_digits(return_X_y=True)
fa = FactorAnalysis(n_components=56, random_state=0)
X_transformed = fa.fit_transform(X)
print(X_transformed.shape)
print(fa.components_.shape)
print(fa.noise_variance_.shape)
print(fa.mean_.shape)
# 变换的公式满足下面这个:
print(np.round(fa.mean_ + np.matmul(X_transformed[0], fa.components_) + fa.noise_variance_, 0))
print(X[0])

(1797, 56)
(56, 64)
(64,)
(64,)
[ 0.  0.  6. 14. 10.  2.  1.  0. -0.  1. 14. 15. 11. 15.  6.  0.  0.  4.
 16.  3.  1. 12.  8.  0. -0.  5. 13.  1.  1.  9.  9. -0.  0.  6.  9.  1.
  1. 10.  9.  0.  0.  5. 12.  1.  2. 13.  7.  0.  0.  2. 15.  6. 11. 13.
  1.  0. -0.  0.  7. 14. 11.  1.  0.  1.]
[ 0.  0.  5. 13.  9.  1.  0.  0.  0.  0. 13. 15. 10. 15.  5.  0.  0.  3.
 15.  2.  0. 11.  8.  0.  0.  4. 12.  0.  0.  8.  8.  0.  0.  5.  8.  0.
  0.  9.  8.  0.  0.  4. 11.  0.  1. 12.  7.  0.  0.  2. 14.  5. 10. 12.
  0.  0.  0.  0.  6. 13. 10.  0.  0.  0.]
